# Imports


In [2]:
##!pip install dash
import pandas as pd
from dash import Dash, html, dcc
import plotly.express as px
from dash.dependencies import Input, Output

# Tratando o Dataframe

In [36]:
df = pd.read_csv('testeDados.csv')
campos_vazios = df.isnull().sum()
mediana_acessos = df['acessos'].median()
data = df.drop(columns=["id_municipio", "sigla_uf", "cnpj"])


In [37]:
df['empresa'].fillna('outra', inplace=True)
df['porte_empresa'].fillna('Pequeno Porte', inplace=True)
df['tecnologia'].fillna('outra', inplace=True)
df['transmissao'].fillna('outra', inplace=True)
df['velocidade'].fillna('512kbps a 2Mbps', inplace=True)
df['acessos'].fillna(mediana_acessos, inplace=True)
df['produto'].fillna('internet', inplace=True)
df = df.dropna()

In [89]:
anos = sorted(df['ano'].unique())
df_media_por_mes = df.groupby(['mes','ano'])['acessos'].mean().reset_index()

Sigla_UF = sorted(df['sigla_uf'].unique())

data_estado = df.groupby(by=["ano", "mes", "empresa", "porte_empresa", "tecnologia", "transmissao", "velocidade","id_municipio", "sigla_uf", "cnpj"], as_index=False).sum()

def get_historico_tecnologia(data, mes):
    historico_tecnologia = data.query(f'ano > 2009 and ano < 2023 and mes == {mes}')
    historico_tecnologia = historico_tecnologia.filter(items=['ano', 'tecnologia', 'acessos'])
    historico_tecnologia = historico_tecnologia.groupby(['ano', 'tecnologia'])['acessos'].sum()
    historico_tecnologia = historico_tecnologia.to_frame().reset_index()

    df_historico_tecnologia = pd.DataFrame()
    for ano in historico_tecnologia['ano'].sort_values().unique():
        df_historico_tecnologia = pd.concat([historico_tecnologia.query('ano == @ano').nlargest(3, 'acessos'), df_historico_tecnologia])

    df_historico_tecnologia = df_historico_tecnologia.sort_values(['ano'])
    df_historico_tecnologia.columns = ['Ano', 'Tecnologia', 'Total de acessos']
    df_historico_tecnologia = df_historico_tecnologia.sort_values(['Ano', 'Total de acessos'])

    return df_historico_tecnologia


In [88]:
app = Dash(__name__)


app.layout = html.Div(children=[
    html.H1(children='Análise dos Dados de Banda Larga', style={'font-family': 'Arial, sans-serif', 'text-align': 'center'}),
    html.Div(children='''
    1- Como está distribuído o número de acessos de acordo com os atributos apresentados?
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '30px','font-size':'20px','font-weight':'bold'}),
    dcc.Dropdown(
        id='ano-dropdown',
        options=[{'label': ano, 'value': ano} for ano in anos], 
        value=anos[0],
        style={'font-family': 'Arial, sans-serif'}
    ),
        html.Div(children=''' ''', style={'padding-bottom': '30px'}),

    dcc.Graph(
        id='graph-content'
    ),
    html.Div(children='''
    2- Quais as características em relação a tecnologia, transmissão e velocidade que predomina hoje em dia no escopo escolhido pelo grupo?
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '30px','font-size':'20px','font-weight':'bold'}),
    
    html.Div(children='''
    Ano:
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '10px' , 'padding-top': '20px'}),

    dcc.Dropdown(
        id='caracteristica-Ano-dropdown',
        options=[{'label': ano, 'value': ano} for ano in anos], 
        value=anos[0],
        style={'font-family': 'Arial, sans-serif'}
    ),

    html.Div(children='''
    Mês:
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '10px', 'padding-top': '20px'}),

    dcc.Dropdown(
        id='caracteristica-MES-dropdown',
        options=[{'label': mes, 'value': mes} for mes in range(1, 13)],
        value=1,
        style={'font-family': 'Arial, sans-serif'}
    ),

    html.Div(children='''
    Sigla UF:
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '10px', 'padding-top': '20px'}),

    dcc.Dropdown(
        id='caracteristica-UF-dropdown',
        options=[{'label': uf, 'value': uf} for uf in Sigla_UF], 
        value=anos[0],
        style={'font-family': 'Arial, sans-serif'}
    ),

    dcc.Graph(
        id='graph-caracteristicas'
    ),
    dcc.Graph(
        id='graph-transmissao'
    ),
    dcc.Graph(
        id='graph-velocidade'
    ),

    html.Div(children='''
    3- Como essas características mudaram ao longo do ano?
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '30px','font-size':'20px','font-weight':'bold'}),

    html.Div(children='''
    Ano:
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '10px' , 'padding-top': '20px'}),

    dcc.Dropdown(
        id='caracteristica-Ano-dropdown-2',
        options=[{'label': ano, 'value': ano} for ano in anos], 
        value=anos[0],
        style={'font-family': 'Arial, sans-serif'}
    ),

    html.Div(children='''
    Mês:
    ''', style={'font-family': 'Arial, sans-serif', 'padding-bottom': '10px', 'padding-top': '20px'}),

    dcc.Dropdown(
        id='caracteristica-MES-dropdown-2',
        options=[{'label': mes, 'value': mes} for mes in range(1, 13)],
        value=1,
        style={'font-family': 'Arial, sans-serif'}
    ),

    dcc.Graph(
        id='graph-historico-tecnologia'
    ),
    
])

@app.callback(
    Output('graph-content', 'figure'),
    [Input('ano-dropdown', 'value')]
)

def update_graph(selected_ano):
    df_ano_selecionado = df_media_por_mes[df_media_por_mes['ano'] == selected_ano]
    fig_ano_mes = px.bar(df_ano_selecionado, x='mes', y='acessos', title=f'Média de Acessos por Mês no Ano de: {selected_ano}', barmode='group')
    
    return fig_ano_mes

# Callback para atualizar o gráfico
@app.callback(
    Output('graph-caracteristicas', 'figure'),
    [Input('caracteristica-Ano-dropdown', 'value'),
     Input('caracteristica-MES-dropdown', 'value'),
     Input('caracteristica-UF-dropdown', 'value')]
)
def update_graph_2(ano, mes, uf):
    analise_tecnologias_recentes = df.query(f'ano == {ano} and mes == {mes} and sigla_uf == "{uf}"')
    analise_tecnologias_recentes = analise_tecnologias_recentes.groupby('tecnologia')['acessos'].sum()
    analise_tecnologias_recentes = pd.DataFrame({'Tecnologia': analise_tecnologias_recentes.index, 'Total de acessos': analise_tecnologias_recentes.values})
    analise_tecnologias_recentes = analise_tecnologias_recentes.sort_values(['Total de acessos'], ascending=False)

    fig = px.bar(analise_tecnologias_recentes, x='Tecnologia', y='Total de acessos', title='Análise de Tecnologias Recentes')
    return fig
# Callback para atualizar o gráfico
@app.callback(
    Output('graph-transmissao', 'figure'),
    [Input('caracteristica-Ano-dropdown', 'value'),
     Input('caracteristica-MES-dropdown', 'value'),]
)
def update_graph_3(ano, mes):
    analise_transmissao_recentes = data_estado.filter(items=['ano', 'mes', 'transmissao', 'acessos'])
    analise_transmissao_recentes = analise_transmissao_recentes.query(f'ano == {ano} and mes == {mes}')
    analise_transmissao_recentes = analise_transmissao_recentes.groupby('transmissao')['acessos'].sum()
    analise_transmissao_recentes = pd.DataFrame({'Transmissão': analise_transmissao_recentes.index, 'Total de acessos': analise_transmissao_recentes.values})
    analise_transmissao_recentes = analise_transmissao_recentes.sort_values(['Total de acessos'], ascending=False)

    fig = px.bar(analise_transmissao_recentes, x='Transmissão', y='Total de acessos', title='Análise de Transmissão Recentes')
    return fig

@app.callback(
    Output('graph-velocidade', 'figure'),
    [Input('caracteristica-Ano-dropdown', 'value'),
     Input('caracteristica-MES-dropdown', 'value'),]
)
def update_graph_4(ano, mes):
    analise_velocidade_recentes = data_estado.filter(items=['ano', 'mes', 'velocidade', 'acessos'])
    analise_velocidade_recentes = analise_velocidade_recentes.query(f'ano == {ano} and mes == {mes}')
    analise_velocidade_recentes = analise_velocidade_recentes.groupby('velocidade')['acessos'].sum()
    analise_velocidade_recentes = pd.DataFrame({'Velocidade': analise_velocidade_recentes.index, 'Total de acessos': analise_velocidade_recentes.values})
    analise_velocidade_recentes = analise_velocidade_recentes.sort_values(['Total de acessos'], ascending=False)

    fig = px.bar(analise_velocidade_recentes, x='Velocidade', y='Total de acessos', title='Análise de Velocidade Recentes')
    return fig

@app.callback(
    Output('graph-historico-tecnologia', 'figure'),
    [Input('caracteristica-Ano-dropdown-2', 'value'),
     Input('caracteristica-MES-dropdown-2', 'value')]
)
def update_graph_5(ano, mes):
    df_historico_tecnologia = get_historico_tecnologia(df,mes)

    # Filtre o DataFrame com base no ano e mês selecionados
    df_historico_tecnologia_filtrado = df_historico_tecnologia[(df_historico_tecnologia['Ano'] == ano)]

    # Crie o gráfico de barras
    fig = px.bar(df_historico_tecnologia_filtrado, x='Tecnologia', y='Total de acessos', title=f'Histórico de Tecnologia para o Ano {ano} e Mês {mes}')
    return fig


if __name__ == '__main__':
    app.run_server(debug=True)

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File c:\python311\Lib\site-packages\pandas\core\indexes\base.py:3790, in Index.get_loc(
    self=Index(['Ano', 'Tecnologia', 'Total de acessos'], dtype='object'),
    key='Mês'
)
   3789 try:
-> 3790     return self._engine.get_loc(casted_key)
        casted_key = 'Mês'
        self = Index(['Ano', 'Tecnologia', 'Total de acessos'], dtype='object')
   3791 except KeyError as err:

File index.pyx:152, in pandas._libs.index.IndexEngine.get_loc()

File index.pyx:181, in pandas._libs.index.IndexEngine.get_loc()

File pandas\_libs\hashtable_class_helper.pxi:7080, in pandas._libs.hashtable.PyObjectHashTable.get_item()

File pandas\_libs\hashtable_class_helper.pxi:7088, in pandas._libs.hashtable.PyObjectHashTable.get_item()

KeyError: 'Mês'

The above exception was the direct cause of the following exception:

KeyError                         

In [35]:
#Como essas características mudaram ao longo do ano de 2011 a 2022 no mês de maio

historico_tecnologia = data.query('ano > 2009 and ano < 2023 and mes == 5')
historico_tecnologia = historico_tecnologia.filter(items=['ano', 'tecnologia', 'acessos'])
historico_tecnologia = historico_tecnologia.groupby(['ano', 'tecnologia'])['acessos'].sum()
historico_tecnologia = historico_tecnologia.to_frame().reset_index()
historico_tecnologia

df_historico_tecnologia = pd.DataFrame()
for ano in historico_tecnologia['ano'].sort_values().unique():
   df_historico_tecnologia = pd.concat([historico_tecnologia.query('ano == @ano').nlargest(3, 'acessos'), df_historico_tecnologia])

df_historico_tecnologia = df_historico_tecnologia.sort_values(['ano'])
df_historico_tecnologia.columns = ['Ano', 'Tecnologia', 'Total de acessos']
df_historico_tecnologia = df_historico_tecnologia.sort_values(['Ano', 'Total de acessos'])
df_historico_tecnologia

,Ano,Tecnologia,Total de acessos
7,2012,Wi-Fi,1033.0
8,2012,xDSL,2655.0
0,2012,Cable Modem,9221.0
11,2013,Fibra,1679.0
12,2013,Wi-Fi,3397.0
13,2013,xDSL,17239.0
20,2014,LTE,355.0
18,2014,Fibra,397.0
23,2014,xDSL,35989.0
28,2015,Wi-Fi,1089.0


In [36]:
#Quais empresas com maior número de contratos considerando o porte da empresa

analise_empresa_grande = data.query('ano == 2021 and mes == 5 and porte_empresa == "Grande Porte"')
analise_empresa_grande = analise_empresa_grande.groupby(['empresa', 'porte_empresa'])['acessos'].sum()
analise_empresa_grande = analise_empresa_grande.to_frame().reset_index()
analise_empresa_grande = analise_empresa_grande.sort_values('acessos', ascending=False)
analise_empresa_grande

analise_empresa_pequena = data.query('ano == 2021 and mes == 5 and porte_empresa == "Pequeno Porte"')
analise_empresa_pequena = analise_empresa_pequena.groupby(['empresa', 'porte_empresa'])['acessos'].sum()
analise_empresa_pequena = analise_empresa_pequena.to_frame().reset_index()
analise_empresa_pequena = analise_empresa_pequena.sort_values('acessos', ascending=False)
analise_empresa_pequena = analise_empresa_pequena.head(4)
analise_empresa_pequena

analise_porte_empresas = pd.concat([analise_empresa_grande, analise_empresa_pequena])
analise_porte_empresas

,empresa,porte_empresa,acessos
1,OI,Grande Porte,3690.0
3,VIVO,Grande Porte,3258.0
0,CLARO,Grande Porte,1532.0
2,TIM,Grande Porte,15.0
14,ALGAR (CTBC TELECOM),Pequeno Porte,26590.0
370,UNIFIQUE,Pequeno Porte,5881.0
219,MALUGAINFOR COMERCIO DE PRODUTOS DE INFORMATIC...,Pequeno Porte,4257.0
304,QWERTY COMUNICACOES E TECNOLOGIA EDUCACIONAL LTDA,Pequeno Porte,3678.0


In [34]:
# Como o perfil do tipo de empresa mudou ao longo dos anos nos estados? Houve crescimento ou redução em relação a um tipo de porte específico

historico_acessos_porte = data.query('mes == 5 and ano > 2009 and ano <= 2022')
historico_acessos_porte = historico_acessos_porte.filter(['ano', 'mes', 'porte_empresa', 'acessos'])
historico_acessos_porte = historico_acessos_porte.groupby(['ano', 'mes', 'porte_empresa'])['acessos'].sum()
historico_acessos_porte = historico_acessos_porte.to_frame()
historico_acessos_porte = historico_acessos_porte.reset_index()
historico_acessos_porte.columns= ['Ano', 'Mês', 'Porte empresa', 'Total de acessos']
historico_acessos_porte

,Ano,Mês,Porte empresa,Total de acessos
0,2012,5,Grande Porte,11124.0
1,2012,5,Pequeno Porte,2383.0
2,2013,5,Grande Porte,19504.0
3,2013,5,Pequeno Porte,3491.0
4,2014,5,Grande Porte,34142.0
5,2014,5,Pequeno Porte,3242.0
6,2015,5,Grande Porte,74970.0
7,2015,5,Pequeno Porte,8164.0
8,2016,5,Grande Porte,200964.0
9,2016,5,Pequeno Porte,18873.0
